# Patient Cohort Building with NLP and Knowledge Graphs

In this notebook, we will build a Knowledge Graph (KG) using Spark NLP relation extraction models and Neo4j. The main point of this notebook is to show creating a clinical knowledge graph using Spark NLP pretrained models. For this purpose, we will use pretrained relation extraction and NER models. After creating the knowledge graph, we will query the KG to get some insightful results.

[Cluster Setup](https://nlp.johnsnowlabs.com/docs/en/licensed_install#install-on-databricks)

In [0]:
%pip install neo4j

## Creation of the Knowledge Graph

### Neo4j Connection

In [0]:
from neo4j import GraphDatabase
import time
from tqdm import tqdm
import pandas as pd
import json

In [0]:
notes_path='/FileStore/HLS/kg/data/'
patient_df = pd.read_csv(f'{notes_path}/data.csv', sep=';')
patient_df

Out[3]:

,subject_id,date,text,gender,dateOfBirth
0,19823,2167-02-25,Admission Date: [**2167-2-16**] Dischar...,F,2099-05-05
1,19823,2167-11-27,Admission Date: [**2167-11-27**] Discha...,F,2099-05-05
2,19823,2170-10-12,Admission Date: [**2170-9-19**] ...,F,2099-05-05
3,19823,2172-06-22,Admission Date: [**2172-6-13**] ...,F,2099-05-05
4,19823,2167-12-07,PATIENT/TEST INFORMATION:\nIndication: Aortic ...,F,2099-05-05
...,...,...,...,...,...
960,70004,2182-06-14,[**2182-6-14**] 10:45 AM\n MR HEAD W & W/O CON...,M,2127-12-06
961,70004,2182-06-25,FDG TUMOR IMAGING (PET-CT) ...,M,2127-12-06
962,70004,2182-08-05,[**2182-8-5**] 11:46 AM\n MR HEAD W & W/O CONT...,M,2127-12-06
963,70004,2182-08-23,FDG TUMOR IMAGING (PET-CT) ...,M,2127-12-06


In [0]:
patient_demographics = patient_df[['subject_id', 'gender', 'dateOfBirth']].drop_duplicates().reset_index(drop=True)
patient_demographics

Out[4]:

,subject_id,gender,dateOfBirth
0,19823,F,2099-05-05
1,22015,M,2085-10-04
2,17494,F,2193-10-01
3,21153,M,2057-03-03
4,12200,M,2136-06-17
5,23266,M,2122-03-16
6,75632,M,2118-09-04
7,27386,M,2114-08-20
8,28552,F,2044-10-22
9,70004,M,2127-12-06


In [0]:
outname = 'posology_RE_rxnorm_w_drug_resolutions.csv'
outdir = f'{delta_path}/golden/dataset'
pos_RE_result =  pd.read_csv(outdir+outname)
pos_RE_result.head()

Out[5]:

,subject_id,date,relation,entity1,entity1_begin,entity1_end,chunk1,entity2,entity2_begin,entity2_end,chunk2,confidence,rx_text,sent_id,ner_chunk,entity,rxnorm_code,all_codes,resolutions,res,resolution,drug_resolution
0,19823,2167-02-25,DRUG-FORM,DRUG,1391,1399,albuterol,FORM,1414,1423,nebulizers,1.0,Albuterol nebulizers,0,Albuterol nebulizers,NaN,2108226,['2108226' '1154602' '370790' '1154603' '21082...,['albuterol Inhalation Solution' 'albuterol In...,['albuterol,albuterol,albuterol
1,19823,2167-02-25,DRUG-FORM,DRUG,1405,1412,atrovent,FORM,1414,1423,nebulizers,1.0,Atrovent nebulizers,0,Atrovent nebulizers,NaN,2108451,['2108451' '1173573' '379767' '1173576' '24637...,['ipratropium Inhalation Solution [Atrovent]' ...,['ipratropium,ipratropium,ipratropium
2,19823,2167-02-25,STRENGTH-DRUG,STRENGTH,1539,1543,40 mg,DRUG,1551,1555,lasix,1.0,Lasix 40 mg,0,Lasix 40 mg,NaN,200809,['200809' '617319' '103919' '1871459' '201286'...,['furosemide 40 MG Oral Tablet [Lasix]' 'atorv...,['furosemide,furosemide,furosemide
3,19823,2167-02-25,ROUTE-DRUG,ROUTE,1548,1549,iv,DRUG,1551,1555,lasix,1.0,Lasix,0,Lasix,NaN,202991,['202991' '151963' '2256936' '2256930' '104372...,['Lasix' 'Lasma' 'lasmiditan Oral Tablet' 'las...,['Lasix',Lasix',lasix
4,19823,2167-02-25,DRUG-STRENGTH,DRUG,2336,2341,amaryl,STRENGTH,2343,2348,2.0 mg,1.0,Amaryl 2.0 mg,0,Amaryl 2.0 mg,NaN,901295,['901295' '153591' '1310138' '213799' '2399657...,['sodium fluoride 2.2 MG [Ludent]' 'glimepirid...,['sodium,sodium,sodium


In [0]:
outname = 'ner_jsl_slim_results.csv'
outdir = f'{delta_path}/golden/dataset'
ner_DF_result = pd.read_csv(outdir+outname)
ner_DF_result.head()

Out[12]:

,subject_id,date,sentence_id,chunk,begin,end,ner_label
0,19823,2167-02-25,0,Shortness of breath,178,196,Symptom
1,19823,2167-02-25,0,cough,199,203,Symptom
2,19823,2167-02-25,1,diabetes type II,345,360,Disease_Syndrome_Disorder
3,19823,2167-02-25,1,congestive heart failure,363,386,Disease_Syndrome_Disorder
4,19823,2167-02-25,1,hypertension,413,424,Disease_Syndrome_Disorder


In [0]:
class Neo4jConnection:
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, parameters=None, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query, parameters))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response

In [0]:
uri = 'bolt://44.195.22.173:7687' 
pwd = 'hatchets-oxygen-tachometer'
user = 'neo4j'

conn = Neo4jConnection(uri=uri, user=user , pwd=pwd)

**Creating constraints:**

In [0]:
conn.query('CREATE CONSTRAINT patients IF NOT EXISTS ON (p:Patient) ASSERT p.name IS UNIQUE;')
conn.query('CREATE CONSTRAINT rx_norm_codes IF NOT EXISTS ON (rx:RxNorm) ASSERT rx.code IS UNIQUE;')
conn.query('CREATE CONSTRAINT drugs IF NOT EXISTS ON (drug:Drug) ASSERT drug.name IS UNIQUE;')
conn.query('CREATE CONSTRAINT ners IF NOT EXISTS ON (n:NER) ASSERT n.name IS UNIQUE;')
conn.query('CREATE CONSTRAINT symptoms IF NOT EXISTS ON (s:Symptom) ASSERT s.name IS UNIQUE;')
conn.query('CREATE CONSTRAINT bodyParts IF NOT EXISTS ON (bp:BodyPart) ASSERT bp.name IS UNIQUE;')
conn.query('CREATE CONSTRAINT procedures IF NOT EXISTS ON (p:Procedure) ASSERT p.name IS UNIQUE;')
conn.query('CREATE CONSTRAINT tests IF NOT EXISTS ON (t:Test) ASSERT t.name IS UNIQUE;')
conn.query('CREATE CONSTRAINT dsds IF NOT EXISTS ON (dsd:DSD) ASSERT dsd.name IS UNIQUE;')

Out[7]: []

**defining helper functions:**

In [0]:
def update_data(query, rows, batch_size = 10000):
    total = 0
    batch = 0
    start = time.time()
    result = None
    while batch * batch_size < len(rows):
        res = conn.query(query, parameters={'rows': rows[batch*batch_size:(batch+1)*batch_size].to_dict('records')})
        total += res[0]['total']
        batch += 1
        result = {"total":total, "batches":batch, "time":time.time()-start}
        print(result)
    return result

In [0]:
def add_patients(rows, batch_size=10000):
    query = '''
    UNWIND $rows as row
    MERGE(p:Patient{name:row.subject_id}) 
    ON CREATE SET p.gender      = row.gender,
                  p.dateOfBirth = row.dateOfBirth

    WITH p
    MATCH (p)
    RETURN count(*) as total
    '''
    return update_data(query, rows, batch_size)

add_patients(patient_demographics)

{'total': 10, 'batches': 1, 'time': 0.11297821998596191}
Out[9]: {'total': 10, 'batches': 1, 'time': 0.11297821998596191}

In [0]:
def add_drugs_ners(rows, batch_size=1000):
    query = '''
    UNWIND $rows as row
    
    MERGE(p:Patient{name:row.subject_id}) 
    MERGE(rx:RxNorm{code:row.rxnorm_code})
    MERGE (p)-[:RXNORM_CODE{date:date(row.date)}]->(rx)
    
    MERGE (d:Drug{name:row.drug_resolution})
    MERGE (rx)-[:DRUG_GENERIC{date:date(row.date), patient_name:row.subject_id}]->(d)
    
    MERGE(n1:NER{name:row.chunk1}) ON CREATE SET n1.type=row.entity1
    MERGE(n2:NER{name:row.chunk2}) ON CREATE SET n2.type=row.entity2
    
    WITH *
    MATCH (d:Drug{name:row.drug_resolution}), (n1:NER{name:row.chunk1}), (n2:NER{name:row.chunk2})
    CALL apoc.create.relationship (d,row.entity1, {patient_name:row.subject_id, date:date(row.date)}, n1) YIELD rel as relx
    CALL apoc.create.relationship (d,row.entity2, {patient_name:row.subject_id, date:date(row.date)}, n2) YIELD rel as rely
    
    WITH d
    MATCH (d)
    RETURN count(*) as total  
    '''
    return update_data(query, rows, batch_size)
  
add_drugs_ners(pos_RE_result)

{'total': 1000, 'batches': 1, 'time': 4.331569671630859}
{'total': 2000, 'batches': 2, 'time': 9.040156364440918}
{'total': 2782, 'batches': 3, 'time': 11.14558482170105}
Out[10]: {'total': 2782, 'batches': 3, 'time': 11.14558482170105}

**splitting dataframe into multiple dataframes by ner_label and creating nodes and relationships**

In [0]:
# spliting dataframe into multiple dataframe by ner_label
grouped        = ner_DF_result.groupby('ner_label')
df_symptom     = grouped.get_group('Symptom')
df_dsd         = grouped.get_group('Disease_Syndrome_Disorder')
df_test        = grouped.get_group('Test')
df_bodyPart    = grouped.get_group('Body_Part')
df_procedure   = grouped.get_group('Procedure')

In [0]:
def add_symptoms(rows, batch_size=500):
    query = '''
    UNWIND $rows as row
    MATCH(p:Patient{name:row.subject_id})
    MERGE(n:Symptom {name:row.chunk})
    MERGE (p)-[:IS_SYMPTOM{date:date(row.date)}]->(n)

    WITH n
    MATCH (n)
    RETURN count(*) as total  
    '''
    return update_data(query, rows, batch_size)
  
add_symptoms(df_symptom)

{'total': 500, 'batches': 1, 'time': 0.4843027591705322}
{'total': 1000, 'batches': 2, 'time': 0.8330910205841064}
{'total': 1500, 'batches': 3, 'time': 1.1354827880859375}
{'total': 2000, 'batches': 4, 'time': 1.3480288982391357}
{'total': 2500, 'batches': 5, 'time': 1.5601437091827393}
{'total': 3000, 'batches': 6, 'time': 1.862185001373291}
{'total': 3500, 'batches': 7, 'time': 2.0643417835235596}
{'total': 4000, 'batches': 8, 'time': 2.2615597248077393}
{'total': 4500, 'batches': 9, 'time': 2.532264471054077}
{'total': 5000, 'batches': 10, 'time': 2.7331066131591797}
{'total': 5036, 'batches': 11, 'time': 2.8320295810699463}
Out[14]: {'total': 5036, 'batches': 11, 'time': 2.8320295810699463}

In [0]:
def add_dsds(rows, batch_size=500):
    query = '''
    UNWIND $rows as row
    MATCH(p:Patient{name:row.subject_id})
    MERGE(n:DSD {name:row.chunk})
    MERGE (p)-[:IS_DSD{date:date(row.date)}]->(n)

    WITH n
    MATCH (n)
    RETURN count(*) as total  
    '''
    return update_data(query, rows, batch_size)
  
add_dsds(df_dsd)

{'total': 500, 'batches': 1, 'time': 0.5363547801971436}
{'total': 1000, 'batches': 2, 'time': 0.7881300449371338}
{'total': 1500, 'batches': 3, 'time': 0.971163272857666}
{'total': 2000, 'batches': 4, 'time': 1.10390305519104}
{'total': 2500, 'batches': 5, 'time': 1.4679605960845947}
{'total': 2798, 'batches': 6, 'time': 1.7660226821899414}
Out[15]: {'total': 2798, 'batches': 6, 'time': 1.7660226821899414}

In [0]:
def add_tests(rows, batch_size=500):
    query = '''
    UNWIND $rows as row
    MATCH(p:Patient{name:row.subject_id})
    MERGE(n:Test {name:row.chunk})
    MERGE (p)-[:IS_TEST{date:date(row.date)}]->(n)

    WITH n
    MATCH (n)
    RETURN count(*) as total  
    '''
    return update_data(query, rows, batch_size)
  
add_tests(df_test)

{'total': 500, 'batches': 1, 'time': 0.39699244499206543}
{'total': 1000, 'batches': 2, 'time': 0.6146259307861328}
{'total': 1500, 'batches': 3, 'time': 1.009399175643921}
{'total': 2000, 'batches': 4, 'time': 1.292588233947754}
{'total': 2500, 'batches': 5, 'time': 1.4985682964324951}
{'total': 3000, 'batches': 6, 'time': 1.6966218948364258}
{'total': 3196, 'batches': 7, 'time': 1.8073601722717285}
Out[16]: {'total': 3196, 'batches': 7, 'time': 1.8073601722717285}

In [0]:
def add_bodyParts(rows, batch_size=500):
    query = '''
    UNWIND $rows as row
    MATCH(p:Patient{name:row.subject_id})
    MERGE(n:BodyPart {name:row.chunk})
    MERGE (p)-[:IS_BODYPART{date:date(row.date)}]->(n)

    WITH n
    MATCH (n)
    RETURN count(*) as total
    '''
    return update_data(query, rows, batch_size)
  
add_bodyParts(df_bodyPart)

{'total': 500, 'batches': 1, 'time': 0.45174646377563477}
{'total': 1000, 'batches': 2, 'time': 0.6665709018707275}
{'total': 1500, 'batches': 3, 'time': 0.928584098815918}
{'total': 2000, 'batches': 4, 'time': 1.1381957530975342}
{'total': 2500, 'batches': 5, 'time': 1.263178825378418}
{'total': 3000, 'batches': 6, 'time': 1.4509198665618896}
{'total': 3500, 'batches': 7, 'time': 1.5821709632873535}
{'total': 4000, 'batches': 8, 'time': 1.7095832824707031}
{'total': 4106, 'batches': 9, 'time': 1.7992794513702393}
Out[17]: {'total': 4106, 'batches': 9, 'time': 1.7992794513702393}

In [0]:
def add_procedures(rows, batch_size=500):
    query = '''
    UNWIND $rows as row
    MATCH(p:Patient{name:row.subject_id})
    MERGE(n:Procedure {name:row.chunk})
    MERGE (p)-[:IS_PROCEDURE{date:date(row.date)}]->(n)

    WITH n
    MATCH (n)
    RETURN count(*) as total  
    '''
    return update_data(query, rows, batch_size)
  
add_procedures(df_procedure)

{'total': 500, 'batches': 1, 'time': 0.25766968727111816}
{'total': 1000, 'batches': 2, 'time': 0.41423821449279785}
{'total': 1385, 'batches': 3, 'time': 0.530022144317627}
Out[18]: {'total': 1385, 'batches': 3, 'time': 0.530022144317627}

In [0]:
query_string = '''
CALL db.labels() YIELD label
CALL apoc.cypher.run('MATCH (:`'+label+'`) RETURN count(*) as count',{}) YIELD value
RETURN label, value.count as size
'''
df_nodes = pd.DataFrame([dict(_) for _ in conn.query(query_string)])
df_nodes

Out[19]:

,label,size
0,Patient,10
1,RxNorm,1068
2,Drug,680
3,NER,897
4,Symptom,2609
5,BodyPart,1205
6,Procedure,642
7,Test,1248
8,DSD,1117


In [0]:
query_string = '''
CALL db.relationshipTypes() YIELD relationshipType as type
CALL apoc.cypher.run('MATCH ()-[:`'+type+'`]->() RETURN count(*) as count',{}) YIELD value
RETURN type, value.count as size
'''
df_relationships = pd.DataFrame([dict(_) for _ in conn.query(query_string)])
df_relationships

Out[20]:

,type,size
0,RXNORM_CODE,1665
1,DRUG_GENERIC,1665
2,DRUG,2782
3,FORM,241
4,STRENGTH,791
5,ROUTE,930
6,FREQUENCY,487
7,DOSAGE,320
8,DURATION,13
9,IS_SYMPTOM,4080


**database schema visualization**

<img src="https://raw.githubusercontent.com/iamvarol/blogposts/main/databricks/images/db_viz.png">

## Queries

**patient 21153's prescriptions:**

In [0]:
patient_name = '21153'
query_part1 = 'MATCH (p:Patient)-[rel_rx]->(rx:RxNorm)-[rel_d]->(d:Drug)-[rel_n]->(n:NER) ' #  
query_part2 = f'WHERE p.name ={patient_name} AND rel_n.date=rel_rx.date AND rel_n.patient_name=p.name ' # 
query_part3 = '''RETURN DISTINCT
                 p.name as patient_name,
                 rel_rx.date as date,
                 d.name as drug_generic_name,  
                 rx.code as rxnorm_code,
                 COALESCE(n.name,'') +  "(" + COALESCE (type(rel_n), "") + ")" as details
                 '''
query_string = query_part1 + query_part2 + query_part3

df = pd.DataFrame([dict(_) for _ in conn.query(query_string)])
df = df.drop_duplicates(subset= ['patient_name', 'date', 'drug_generic_name'])
df = df.groupby(['patient_name', 'date', 'drug_generic_name', 'rxnorm_code']).agg(lambda x: ' '.join(x)).reset_index()
df

Out[21]:

,patient_name,date,drug_generic_name,rxnorm_code,details
0,21153,2109-12-17,dex4,607868,mso4(DRUG)
1,21153,2109-12-17,everolimus,1310138,2 units(DOSAGE)
2,21153,2109-12-18,cidaflex,1013644,po(ROUTE)
3,21153,2109-12-18,clorsulon,1006411,po(ROUTE)
4,21153,2109-12-18,crolom,216281,hcl(DRUG)
...,...,...,...,...,...
130,21153,2110-10-20,cotab,799044,contrast(DRUG)
131,21153,2110-10-20,dibucaine,335534,1%(STRENGTH)
132,21153,2110-10-20,ofev,1592743,contrast(DRUG)
133,21153,2110-12-02,10,1807637,10 cc(DOSAGE)


**patient 21153's journey in medical records:symptoms, procedures, disease-syndrome-disorders, test, drugs & rxnorms**

In [0]:
patient_name = '21153'

query_part1 = f'MATCH (p:Patient)-[r1:IS_SYMPTOM]->(s:Symptom) WHERE p.name = {patient_name} '
query_part2 = '''
WITH DISTINCT p.name as patients, r1.date as dates, COLLECT(DISTINCT s.name) as symptoms, COUNT(DISTINCT s.name) as num_symptoms

MATCH (p:Patient)-[r2:IS_PROCEDURE]->(pr:Procedure)
WHERE p.name=patients AND r2.date = dates

WITH DISTINCT p.name as patients, r2.date as dates, COLLECT(DISTINCT pr.name) as procedures, COUNT(DISTINCT pr.name) as num_procedures, symptoms, num_symptoms
MATCH (p:Patient)-[r3:IS_DSD]->(_d:DSD) 
WHERE p.name=patients AND r3.date = dates

WITH DISTINCT p.name as patients, r3.date as dates, symptoms, num_symptoms, procedures, num_procedures,  COLLECT(DISTINCT _d.name) as dsds, COUNT(DISTINCT _d.name) as num_dsds
MATCH (p:Patient)-[r4:IS_TEST]->(_t:Test) 
WHERE p.name=patients AND r4.date = dates

WITH DISTINCT p.name as patients, r4.date as dates, symptoms, num_symptoms, procedures, num_procedures, dsds, num_dsds, COLLECT(_t.name) as tests, COUNT(_t.name) as num_tests
MATCH (p:Patient)-[r5:RXNORM_CODE]->(rx:RxNorm)-[r6]->(_d:Drug)
WHERE p.name=patients AND r5.date = dates
RETURN DISTINCT p.name as patients, r5.date as dates, symptoms, num_symptoms, procedures, num_procedures, dsds, num_dsds, tests, num_tests, COLLECT(DISTINCT toLower(_d.name)) as drugs, COUNT(DISTINCT toLower(_d.name)) as num_drugs, COLLECT(DISTINCT rx.code) as rxnorms, COUNT(DISTINCT rx.code) as num_rxnorm
ORDER BY dates;
'''
query_string = query_part1 + query_part2
df = pd.DataFrame([dict(_) for _ in conn.query(query_string)])
df

Out[22]:

,patients,dates,symptoms,num_symptoms,procedures,num_procedures,dsds,num_dsds,tests,num_tests,drugs,num_drugs,rxnorms,num_rxnorm
0,21153,2109-12-17,"[enviromental - rash, unresponsive, flatus, pa...",18,"[thyraloglossal cyst removal, serosagiuos drai...",9,"[esophgeal varices, cirrhosis, METABOLIC ALKAL...",7,"[MAINTAIN FSBG, SvO2, PA, POST PLT CT, INR REM...",18,"[everolimus, dex4]",2,"[1310138, 607868]",2
1,21153,2109-12-18,"[pain, Weak gag, sob, Pain, slight abd pain, s...",19,"[Suction, removal of swan ganz catheter, Liver...",4,"[ERYTHEMA, DSD, FORGETFUL AT TIMES, PH DOWN, D...",10,"[ABG, glucose, magnesium, ABG'S CLOSELY, Repla...",12,"[dex4, insulin, clorsulon, cidaflex, crolom]",5,"[607868, 1740938, 1006411, 1013644, 216281]",5
2,21153,2109-12-19,"[aspiration voice clear after swallowing, slig...",25,"[Anticipate extubation, Tolerating extubation,...",4,"[metabolic alkalosis, Assit with pulmonary tio...",6,"[IVP, 7p-7a, ABG - metabolic alkolotic, LS CTA...",9,"[tobi, dimethicone, clonidine, diamox, insulin...",6,"[220329, 1310821, 1360120, 151619, 139825, 211...",6
3,21153,2109-12-20,"[C/O SOB, SOB, N/V]",3,"[INCISION - STAPLES INTACT, LUNGS CTA BILAT]",2,"[MIN C/O PAIN, DIMINISHED AT BASES, ABG'S ACCE...",3,"[LS CTA, INCLUDING PLT CT, MIN ASSIST, HCL GTT...",6,[x-seb],1,[220956],1
4,21153,2109-12-21,"[flatus, Flat affect, nonproductive cough, bm,...",17,[bile drainage],1,[Hypertensive],1,"[cyclosporin levels, HRR, Plt, Phos, Hct, RA, ...",9,[insulin],1,[378841],1
5,21153,2109-12-28,"[estimated\nblood loss, acute\ndistress, pain,...",15,"[tonsillectomy, extubation, liver transplant, ...",7,"[numerous spider nevi, chronic hepatitis\nC, t...",9,"[T tube study, platelet count, liver function ...",32,"[percocet, p2e1, citalopram, pantoprazole, cal...",36,"[42844, 219017, 329444, 330396, 369633, 2599, ...",46
6,21153,2110-01-03,"[opacification, ductal dilatation]",2,[liver transplant],1,"[LIVER TX,ABD PAIN, stricture]",2,"[LFTs, fluoroscopic\n guidance]",2,"[slow, dibucaine]",2,"[154995, 3339]",2
7,21153,2110-01-14,"[obstruction, mild intrahepatic biliary dilata...",10,"[liver transplant, transplant liver]",2,"[abcess, Hematoma, biloma, Mid common bile\n d...",6,"[pulsed Doppler images, eval liver perfusion, ...",6,[lisinopril],1,[201382],1
8,21153,2110-01-15,[drainage],1,"[drainage, placement of 8.5 Fr percutaneous tr...",9,"[stricture, LIVER TRANSPLANT, ampullary strict...",10,"[Tube cholangiogram, Cholangiogram, cholangiog...",3,[lidocaine],1,[439728],1
9,21153,2110-01-19,"[several prominent lymph nodes, Large subscapu...",14,"[percutaneous tube placement, PTC drain placem...",2,"[ascites, LIVER TRANSPLANT, PTC PLACEMENT, bil...",4,"[Contrast-enhanced CT, CT 100CC, CT OF THE PEL...",5,"[acalabrutinib, docusate, cotab, ofev]",4,"[1986815, 1302275, 799044, 1592743]",4


<img src="https://raw.githubusercontent.com/iamvarol/blogposts/main/databricks/images/patients_journey.png">

**which patient used isosorbide:**

In [0]:
# drug based query
drug_generic_name = 'isosorbide' 

query_part1 = 'MATCH (p:Patient)-[rel_rx]->(rx:RxNorm)-[rel_d]->(d:Drug)-[rel_n]->(n:NER) '
query_part2 = f'WHERE d.name ="{drug_generic_name}" AND rel_n.date=rel_rx.date AND rel_n.patient_name=p.name '
query_part3 = '''RETURN DISTINCT
                 d.name as drug_generic_name, 
                 p.name as patient_name, 
                 rel_rx.date as date, 
                 rx.code as rxnorm_code, 
                 COALESCE(n.name,'') +  "(" + COALESCE (type(rel_n), "") + ")" as details'''

query_string = query_part1 + query_part2 + query_part3
df = pd.DataFrame([dict(_) for _ in conn.query(query_string)])
df = df.groupby(['patient_name', 'date', 'rxnorm_code','drug_generic_name']).agg(lambda x : ' '.join(x)).reset_index()
df

Out[23]:

,patient_name,date,rxnorm_code,drug_generic_name,details
0,12200,2136-06-20,153689,isosorbide,piv(DRUG) 30cc/kg(STRENGTH)
1,19823,2167-12-01,201438,isosorbide,60meq(DRUG) x1(FREQUENCY) po(ROUTE)
2,19823,2172-06-20,201415,isosorbide,medicated(DRUG) 5 mg(STRENGTH)
3,21153,2110-09-09,153689,isosorbide,contrast(DRUG) 30 cc(DOSAGE)
4,22015,2161-05-05,153689,isosorbide,30 meq(STRENGTH) kphos(DRUG)


**patients who are prescribed Lasix between May 2060 and May 2125:**

In [0]:
query_string ='''
MATCH (p:Patient)-[rel_rx]->(rx:RxNorm)-[rel_d]->(d:Drug)-[rel_n:DRUG]->(n:NER)
WHERE d.name IN ['lasix']
      AND rel_n.patient_name=p.name
      AND rel_n.date=rel_rx.date 
      AND rel_rx.date >= date("2060-05-01")
      AND rel_n.date >= date("2060-05-01")
      AND rel_rx.date < date("2125-05-01")
      AND rel_n.date < date("2125-05-01")
RETURN DISTINCT
      d.name as drug_generic_name, 
      p.name as patient_name, 
      rel_rx.date as date
ORDER BY date ASC
'''

df = pd.DataFrame([dict(_) for _ in conn.query(query_string)])
df

Out[24]:

,drug_generic_name,patient_name,date
0,lasix,28552,2122-07-17
1,lasix,28552,2122-07-18
2,lasix,28552,2122-07-20
3,lasix,28552,2122-07-21
4,lasix,28552,2122-07-22
5,lasix,28552,2122-07-24
6,lasix,28552,2122-07-29


<img src="https://raw.githubusercontent.com/iamvarol/blogposts/main/databricks/images/lasix.png">

**patients using warfarin 2mg and up:**

In [0]:
query_string ='''
MATCH (p:Patient)-[rel_rx]->(rx:RxNorm)-[rel_d]->(d:Drug)-[rel_n:STRENGTH]->(n:NER)
WHERE toLower(d.name)='warfarin'
      AND rel_n.patient_name=p.name
      AND rel_n.date=rel_rx.date 
      AND toInteger(left(n.name,1)) >=2
RETURN  DISTINCT
      d.name as drug_generic_name,
      rx.code as rxnorm_code,
      p.name as patient_name,
      n.name as strength,
      rel_rx.date as date
'''
df = pd.DataFrame([dict(_) for _ in conn.query(query_string)])
df

Out[25]:

,drug_generic_name,rxnorm_code,patient_name,strength,date
0,warfarin,855313,19823,2.5 mg,2167-11-27


**dangerous drug combinations:**

In [0]:
query_string ='''
WITH ["ibuprofen", "naproxen", "diclofenac", "indometacin", "ketorolac", "aspirin", "ketoprofen", "dexketoprofen", "meloxicam"] AS nsaids
MATCH (p:Patient)-[r1:RXNORM_CODE]->(rx:RxNorm)-[r2]->(d:Drug)
WHERE any(word IN nsaids WHERE d.name CONTAINS word) 
WITH DISTINCT p.name as patients, COLLECT(DISTINCT d.name) as nsaid_drugs, COUNT(DISTINCT d.name) as num_nsaids
MATCH (p:Patient)-[r1:RXNORM_CODE]->(rx:RxNorm)-[r2]->(d:Drug)
WHERE p.name=patients AND d.name='warfarin'
RETURN DISTINCT patients, 
                nsaid_drugs, 
                num_nsaids, 
                d.name as warfarin_drug, 
                r1.date as date
'''

df = pd.DataFrame([dict(_) for _ in conn.query(query_string)])
df

Out[26]:

,patients,nsaid_drugs,num_nsaids,warfarin_drug,date
0,19823,"[diclofenac, aspirin, ketoprofen]",3,warfarin,2172-06-22
1,19823,"[diclofenac, aspirin, ketoprofen]",3,warfarin,2167-11-27
2,19823,"[diclofenac, aspirin, ketoprofen]",3,warfarin,2170-10-12


<img src="https://raw.githubusercontent.com/iamvarol/blogposts/main/databricks/images/ddc.png">

**patients who underwent a hernia repair or appendectomy, or cholecystectomy:**

In [0]:
query_string = """
MATCH (pcd1:Procedure)-[rel1:IS_PROCEDURE]-(pati1:Patient)
WHERE pcd1.name CONTAINS 'hernia repair' OR pcd1.name CONTAINS 'appendectomy' OR pcd1.name CONTAINS 'cholecystectomy'
RETURN DISTINCT pati1.name as patients, 
                COLLECT(DISTINCT toLower(pcd1.name)) as procedures
"""

df = pd.DataFrame([dict(_) for _ in conn.query(query_string)])
df

Out[27]:

,patients,procedures
0,21153,"[hernia repair, appendectomy]"


In [0]:
# patients with chest pain and shortness of breath
query_string = """
MATCH (p1:Patient)-[r1:IS_SYMPTOM]->(s1:Symptom),
(p2:Patient)-[r2:IS_SYMPTOM]->(s2:Symptom)
WHERE s1.name CONTAINS "chest pain" AND s2.name CONTAINS "shortness of breath"
    AND p2.name=p1.name AND r2.date = r1.date
RETURN DISTINCT p1.name as patient, r1.date as date,s1.name as symptom1, s2.name as symptom2
ORDER BY patient
"""
df = pd.DataFrame([dict(_) for _ in conn.query(query_string)])
df

Out[28]:

,patient,date,symptom1,symptom2
0,19823,2170-10-12,chest pain,shortness of breath
1,19823,2167-02-25,chest pain,shortness of breath


**patients with hypertension or diabetes with chest pain:**

In [0]:
query_string = """
MATCH (p:Patient)-[r:IS_SYMPTOM]->(s:Symptom),
(p1:Patient)-[r2:IS_DSD]->(_dsd:DSD)
WHERE s.name CONTAINS "chest pain" AND p1.name=p.name AND _dsd.name IN ['hypertension', 'diabetes'] AND r2.date=r.date
RETURN DISTINCT p.name as patient, r.date as date, _dsd.name as dsd, s.name as symptom
"""
df = pd.DataFrame([dict(_) for _ in conn.query(query_string)])
df

Out[29]:

,patient,date,dsd,symptom
0,27386,2189-07-06,hypertension,chest pain
1,19823,2167-02-25,hypertension,chest pain


<img src="https://raw.githubusercontent.com/iamvarol/blogposts/main/databricks/images/chest_pain.png">

## License
Copyright / License info of the notebook. Copyright [2021] the Notebook Authors.  The source in this notebook is provided subject to the [Apache 2.0 License](https://spdx.org/licenses/Apache-2.0.html).  All included or referenced third party libraries are subject to the licenses set forth below.

|Library Name|Library License|Library License URL|Library Source URL|
| :-: | :-:| :-: | :-:|
|Pandas |BSD 3-Clause License| https://github.com/pandas-dev/pandas/blob/master/LICENSE | https://github.com/pandas-dev/pandas|
|Numpy |BSD 3-Clause License| https://github.com/numpy/numpy/blob/main/LICENSE.txt | https://github.com/numpy/numpy|
|Neo4j |Apache License 2.0|https://github.com/neo4j/neo4j/blob/4.4/LICENSE.txt|https://github.com/neo4j/neo4j|
|Apache Spark |Apache License 2.0| https://github.com/apache/spark/blob/master/LICENSE | https://github.com/apache/spark/tree/master/python/pyspark|
|BeautifulSoup|MIT License|https://www.crummy.com/software/BeautifulSoup/#Download|https://www.crummy.com/software/BeautifulSoup/bs4/download/|
|Requests|Apache License 2.0|https://github.com/psf/requests/blob/main/LICENSE|https://github.com/psf/requests|
|Spark NLP Display|Apache License 2.0|https://github.com/JohnSnowLabs/spark-nlp-display/blob/main/LICENSE|https://github.com/JohnSnowLabs/spark-nlp-display|
|Spark NLP |Apache License 2.0| https://github.com/JohnSnowLabs/spark-nlp/blob/master/LICENSE | https://github.com/JohnSnowLabs/spark-nlp|
|Spark NLP for Healthcare|[Proprietary license - John Snow Labs Inc.](https://www.johnsnowlabs.com/spark-nlp-health/) |NA|NA|




|Author|
|-|
|Databricks Inc.|
|John Snow Labs Inc.|

## Disclaimers
Databricks Inc. (“Databricks”) does not dispense medical, diagnosis, or treatment advice. This Solution Accelerator (“tool”) is for informational purposes only and may not be used as a substitute for professional medical advice, treatment, or diagnosis. This tool may not be used within Databricks to process Protected Health Information (“PHI”) as defined in the Health Insurance Portability and Accountability Act of 1996, unless you have executed with Databricks a contract that allows for processing PHI, an accompanying Business Associate Agreement (BAA), and are running this notebook within a HIPAA Account.  Please note that if you run this notebook within Azure Databricks, your contract with Microsoft applies.